# Transfer Learning on Cats & Dogs with Pretrained ResNet
### - Backbone: ResNet50 (ImageNet pretrained)
### - Epochs: 20
### - Saves: best weights, latest weights, full SavedModel, .keras

In [1]:
import os, random
import numpy as np
import tensorflow as tf
from dataclasses import dataclass
import matplotlib.pyplot as plt

# 0) Reproducibility & Device

In [2]:
def fix_seed(seed=42):
    random.seed(seed); np.random.seed(seed); tf.random.set_seed(seed)
fix_seed(42)

In [3]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
        print("✅ GPU detected:", tf.config.experimental.get_device_details(gpus[0]).get("device_name", "GPU"))
    except Exception as e:
        print("⚠️ Could not enable memory growth:", e)
else:
    print("⚠️ No GPU detected. Using CPU.")

✅ GPU detected: Tesla T4


# 1) Config

In [21]:
@dataclass
class CFG:
    img_size: int = 224
    batch_size: int = 32
    epochs: int = 20              # 요청사항: 20 epoch
    lr: float = 1e-3
    out_dir: str = "/content/ckpt_tf/resnet_transfer"
    use_data_augmentation: bool = True
    freeze_backbone: bool = True  # 첫 단계: backbone freeze
CFG = CFG()
os.makedirs(CFG.out_dir, exist_ok=True)

# 1) Dataset (cats & dogs)

In [ ]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
!unzip -qq cats_and_dogs_filtered.zip

In [6]:
!ls

cats_and_dogs_filtered	cats_and_dogs_filtered.zip  ckpt_tf  sample_data


In [7]:
base_dir = os.path.join("/content/", "cats_and_dogs_filtered")
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "validation")

In [8]:
print(val_dir)

/content/cats_and_dogs_filtered/validation


In [9]:
!ls /content/cats_and_dogs_filtered/train

cats  dogs


In [10]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(CFG.img_size, CFG.img_size),
    batch_size=CFG.batch_size,
    shuffle=True,
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(CFG.img_size, CFG.img_size),
    batch_size=CFG.batch_size,
    shuffle=False,
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [11]:
class_names = train_ds.class_names  # ['cats', 'dogs']
print("Class names:", class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)

Class names: ['cats', 'dogs']


# 3) Model (ResNet50 backbone)

In [12]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess

In [13]:
# Data Augmentation (ON/OFF)
aug_layers = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.02),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name="augment") if CFG.use_data_augmentation else tf.keras.Sequential(name="no_aug")

In [ ]:
# Backbone: ResNet50 (pretrained on ImageNet), no top
backbone = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(CFG.img_size, CFG.img_size, 3)
)
backbone.trainable = not CFG.freeze_backbone  # freeze or unfreeze

In [ ]:
inputs = layers.Input(shape=(CFG.img_size, CFG.img_size, 3), name="input_image")
x = aug_layers(inputs)
# ResNet 전처리 (mean/std shift)
x = layers.Lambda(resnet_preprocess, name="preprocess")(x)
x = backbone(x, training=False)  # freeze시 BN 업데이트 방지 위해 training=False
x = layers.GlobalAveragePooling2D(name="gap")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)  # binary classification

model = models.Model(inputs, outputs, name="ResNet50_transfer")
model.summary()

# 4) Compile

In [16]:
opt = tf.keras.optimizers.Adam(learning_rate=CFG.lr)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
metrics = [tf.keras.metrics.BinaryAccuracy(name="acc"), tf.keras.metrics.AUC(name="auc")]

model.compile(optimizer=opt, loss=loss, metrics=metrics)

# 5) Callbacks (save, early stop, lr schedule)

In [17]:
ckpt_best = os.path.join(CFG.out_dir, "best.weights.h5")
ckpt_last = os.path.join(CFG.out_dir, "latest.weights.h5")

cbs = [
    tf.keras.callbacks.ModelCheckpoint(
        ckpt_best, monitor="val_acc", save_best_only=True, save_weights_only=True, verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        ckpt_last, monitor="val_acc", save_best_only=False, save_weights_only=True, verbose=0
    ),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1),
]


# 6) Train (20 epochs)

In [ ]:
hist = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=CFG.epochs,
    callbacks=cbs
)

# 7) Save training curves

In [19]:
plt.figure()
plt.plot(hist.history["loss"], label="train")
plt.plot(hist.history["val_loss"], label="val")
plt.title("Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()
plt.tight_layout(); plt.savefig(os.path.join(CFG.out_dir, "loss.png")); plt.close()

plt.figure()
plt.plot(hist.history["acc"], label="train")
plt.plot(hist.history["val_acc"], label="val")
plt.title("Accuracy")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend()
plt.tight_layout(); plt.savefig(os.path.join(CFG.out_dir, "accuracy.png")); plt.close()


# 8) Save model

In [24]:
!pwd

/content


In [25]:
!ls

cats_and_dogs_filtered	cats_and_dogs_filtered.zip  ckpt_tf  sample_data


In [23]:
print(CFG.out_dir)

/content/ckpt_tf/resnet_transfer


In [26]:

from google.colab import files

def save_trained_model(save_path):
  model.save_weights(save_path)
  print(f"✅ 모델 가중치 저장 완료: {save_path}")

  files.download(save_path)

In [28]:
savedmodel_dir = CFG.out_dir+ "/saved_model.weights.h5"
print("✅ Save:", ckpt_best, ckpt_last, savedmodel_dir)
save_trained_model(savedmodel_dir)  # SavedModel



✅ Saved: ckpt_tf/resnet_transfer/best.weights.h5 ckpt_tf/resnet_transfer/latest.weights.h5 /content/ckpt_tf/resnet_transfer/saved_model.weights.h5
✅ 모델 가중치 저장 완료: /content/ckpt_tf/resnet_transfer/saved_model.weights.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 9) (Optional) Fine-tuning Stage
##    - 백본 상단 일부를 unfreeze 후, 작은 lr로 몇 epoch 추가 학습

In [ ]:
# Example:
# backbone.trainable = True
# for layer in backbone.layers[:-20]:  # 상단 20개 레이어만 학습 (나머지 동결)
#     layer.trainable = False
# model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=loss, metrics=metrics)
# hist_ft = model.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=cbs)
# model.save(os.path.join(CFG.out_dir, "resnet_transfer_finetuned.keras"))